Following is based on https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/rock-paper-scissors

In [ ]:
# DO NOT MODIFY (RPS_game.py)

import random

def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])

In [ ]:
# RPS.py
def player(prev_play, opponent_history=[]):
    opponent_history.append(prev_play)

    guess = "R"
    if len(opponent_history) > 2:
        guess = opponent_history[-2]

    return guess

In [ ]:
# Test against all players
play(player, quincy, 1000)
play(player, abbey, 1000)
play(player, kris, 1000)
play(player, mrugesh, 1000)

In [ ]:
import operator
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [ ]:
# Hidden Markov Model for player(), works well for quincy and mrugesh but not for abbey and kris
def rindex(lst, value):    # python should have rindex method for lists, unitl then this is used
    return len(lst) - operator.indexOf(reversed(lst), value) - 1
def markov_distributions_0(opphist):
    ss = ''.join(opphist[rindex(opphist, ''):])    # opphist is entire history of all plays since player() function is defined, luckily each new play appends '' to the opphist, that's why we only consider history from last ''
    rps = ('R', 'P', 'S')
    if opphist[-1] in rps:
        pb = [0.0, 0.0, 0.0]
        pb[rps.index(opphist[-1])] = 1.0
        initial_distribution = tfd.Categorical(probs = np.array(pb))
    else:
        initial_distribution = tfd.Categorical(probs = np.ones(3) / 3.0)
    if ('RR' in ss) or ('RP' in ss) or ('RS' in ss):    # pbR will contain tranistion probability from R to R, P and S
        pbR = np.array([ss.count('RR'), ss.count('RP'), ss.count('RS')])
        pbR = pbR / pbR.sum()
    else:
        pbR = np.ones(3) / 3.0
    if ('PR' in ss) or ('PP' in ss) or ('PS' in ss):    # pbP will contain tranistion probability from P to R, P and S
        pbP = np.array([ss.count('PR'), ss.count('PP'), ss.count('PS')])
        pbP = pbP / pbP.sum()
    else:
        pbP = np.ones(3) / 3.0
    if ('SR' in ss) or ('SP' in ss) or ('SS' in ss):    # pbS will contain tranistion probability from S to R, P and S
        pbS = np.array([ss.count('SR'), ss.count('SP'), ss.count('SS')])
        pbS = pbS / pbS.sum()
    else:
        pbS = np.ones(3) / 3.0
    transition_distribution = tfd.Categorical(probs = np.array([pbR, pbP, pbS]))
    return initial_distribution, transition_distribution
def player(prev_play, opponent_history = []):
    opponent_history.append(prev_play)
    initial_distribution, transition_distribution = markov_distributions_0(opponent_history)
    observation_distribution = tfd.Deterministic(loc = np.array([0.0, 1.0, 2.0]))
    #observation_distribution = tfd.Categorical(probs = np.identity(3))    # ideally we should use Categorical distribution for observations, but it doesn't work because mean() is not yet available for Categorical distributions and mode() is not available at all as of tensorflow_probability version 0.20.1. however, sample() works with Categorical
    model = tfd.HiddenMarkovModel(initial_distribution = initial_distribution, transition_distribution = transition_distribution, observation_distribution = observation_distribution, num_steps = 2)
    rps = ('R', 'P', 'S')
    guess = rps[(1 + int(1.499999999999 * model.mean().numpy()[1])) % 3]    # this is very dubious way but can't see anything better until Categorical observations are possible. and I "believe" stretching [0,2] -> [0,3] and taking int might be better than taking round on [0, 2]
    return guess

In [ ]:
# [FAILED APPROACH] Hidden Markov Model for player(), this was supposed to be an improvement of above but has several problems
# basic idea of the improvement was to not count transitions agains and again but update old values based on new entry in opponent_history
# but there are no easy ways to keep track of how many transitions occured separately for each of R, P and S
# furthermore, there is no significant improvement observed in speed
def rindex(lst, value):
    return len(lst) - operator.indexOf(reversed(lst), value) - 1
def markov_distributions(opphist, dists = [None, None]):    # this will initialize dists when function is first defined and retains latest value in any successive calls. this is dangerous method if not being careful
    rps = ('R', 'P', 'S')
    nhist = len(opphist[rindex(opphist, ''):]) - 1    # opphist is entire history of all plays since player() function is defined, luckily each new play appends '' to the opphist, that's why we only consider history from last ''
    if opphist[-1] in rps:
        nm1 = rps.index(opphist[-1])
        pb = np.zeros(3)
        pb[nm1] = 1.0
        dists[0] = tfd.Categorical(probs = np.array(pb))
    else:
        dists[0] = tfd.Categorical(probs = np.ones(3) / 3.0)
    if (nhist > 1) and (opphist[-1] in rps) and (opphist[-2] in rps):
        nm0 = rps.index(opphist[-2])
        pb = dists[1].probs.numpy()
        pb[nm0, :] = pb[nm0, :] * (nhist - 2.0) / (nhist - 1.0)    # this is NOT correct way. instead of nhist, only number of transitions from nm0 should be taken into account
        pb[nm0, nm1] += 1.0 / (nhist - 1.0)
        dists[1] = tfd.Categorical(probs = pb)
    else:
        dists[1] = tfd.Categorical(probs = np.zeros((3, 3)))
    d10 = dists[1].probs.numpy()[0, :]    # this needs to be done because none of the row of transition_distribution should be all 0.0
    d11 = dists[1].probs.numpy()[1, :]
    d12 = dists[1].probs.numpy()[2, :]
    if all(d10 == 0.0):
        d10 = np.ones(3) / 3.0
    if all(d11 == 0.0):
        d11 = np.ones(3) / 3.0
    if all(d12 == 0.0):
        d12 = np.ones(3) / 3.0
    return [dists[0], tfd.Categorical(probs = np.array([d10, d11, d12]))]
def player(prev_play, opponent_history=[]):
    opponent_history.append(prev_play)
    initial_distribution, transition_distribution = markov_distributions(opponent_history)
    observation_distribution = tfd.Deterministic(loc = np.array([0.0, 1.0, 2.0]))
    #observation_distribution = tfd.Categorical(probs = np.identity(3))    # ideally we should use Categorical distribution for observations, but it doesn't work because mean() is not yet available for Categorical distributions and mode() is not available at all as of tensorflow_probability version 0.20.1. however, sample() works with Categorical
    model = tfd.HiddenMarkovModel(initial_distribution = initial_distribution, transition_distribution = transition_distribution, observation_distribution = observation_distribution, num_steps = 2)
    rps = ('R', 'P', 'S')
    guess = rps[(1 + int(1.499999999999 * model.mean().numpy()[1])) % 3]    # this is very dubious way but can't see anything better until Categorical observations are possible. and I "believe" stretching [0,2] -> [0,3] and taking int might be better than taking round on [0, 2]
    return guess

Couldn't much understand or got to working with pomegranate and hmmlearn libraries for Hidden Markov Model

In [ ]:
# this approach only uses opponent's history and trains the model for the entire history again before each guess
rps = ('R', 'P', 'S')
rps2int = {'R' : 0, 'P' : 1, 'S' : 2, '' : 3}
int2rps = {rps2int[c] : c for c in rps2int}
rps_to_int = lambda hist : [rps2int[c] for c in hist]
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 3, output_dim = 8, mask_zero = False, batch_input_shape = (1, None)),    # converts character to trainable vectors. input_dim is size of vocabulary; output_dim is size of each vector; mask_zero is needed to deal with 0-padding in variable length input
    tf.keras.layers.GRU(units = 64, return_sequences = False, stateful = True),    # units is dimensionality of output space
    #tf.keras.layers.LSTM(units = 16),
    tf.keras.layers.Dense(16, activation = 'relu'),
    #tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.Dense(3)
])
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),    # this loss function works because it's applied across the last dimension of the predictions
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01))
weights_0 = model.get_weights()

def player(prev_play, opponent_history=[]):
    opponent_history.append(prev_play)
    if opponent_history[-1] == '':
        tf.keras.backend.clear_session()
        model.set_weights(weights_0)
        model.reset_metrics()
        model.reset_states()
        opponent_history[:] = []
    nhist = len(opponent_history)
    if nhist > 1:
        if len(opponent_history) == 499:
            print(''.join(opponent_history))
        n = 64
        xs = [rps_to_int(opponent_history[i * n : (i+1) * n]) for i in range(len(opponent_history) // n)]
        xs.append(rps_to_int(opponent_history[-n:]))
        ys = [[0.0, 0.0, 0.0] for _ in xs]
        for i in range(len(xs)):
            xx = xs[i].pop()
            ys[i][xx] = 1.0
            dataset = tf.data.Dataset.from_tensors((np.array([xs[i]]), np.array([ys[i]])))
            model.fit(dataset, epochs = 1, verbose = 0)
        guess = rps[(1 + np.argmax(model.predict(np.array([xs[-1][1:] + [xx]]), verbose = 0))) % 3]
    else:
        guess = np.random.choice(rps)
    return guess
#play(player, quincy, 1000)
play(player, abbey, 500)

In [ ]:
# this is the best working approach. it uses both opponent's and self history. only recent history is used to make the next guess but for several epochs
rps = ('R', 'P', 'S')
rps2int = {'R' : 0, 'P' : 1, 'S' : 2, '' : 3}
int2rps = {rps2int[c] : c for c in rps2int}
rps_to_int = lambda hist : [rps2int[c] for c in hist]

inp0 = tf.keras.Input(shape = (None,), batch_size = 1)
x0 = tf.keras.layers.Embedding(input_dim = 3, output_dim = 2, mask_zero = False)(inp0)
x0 = tf.keras.layers.GRU(units = 1, stateful = True, recurrent_initializer = 'glorot_uniform')(x0)
inp1 = tf.keras.Input(shape = (None,), batch_size = 1)
x1 = tf.keras.layers.Embedding(input_dim = 3, output_dim = 2, mask_zero = False)(inp1)
x1 = tf.keras.layers.GRU(units = 1, stateful = True, recurrent_initializer = 'glorot_uniform')(x1)
x2 = tf.keras.layers.concatenate([x0, x1])
x2 = tf.keras.layers.Dense(units = 3, kernel_initializer = 'glorot_uniform')(x2)
model = tf.keras.Model(inputs = (inp0, inp1), outputs = x2)
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),    # this loss function works because it's applied across the last dimension of the predictions
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005))
weights_0 = model.get_weights()    # will be used to reset the model
nn = enumerate([6, 3, 3, 11])    # how much history to pass while training for each player

def player(prev_play, opponent_history=[], self_history=[], n0 = [6]):
    opponent_history.append(prev_play)
    # this resetting of model is done because main.py code keeps appending opponent history to this list but starts any new play with ''. this also allows to set any custom parameter for a specific player, like how n0 is used here
    if opponent_history[-1] == '':
        tf.keras.backend.clear_session()
        model.set_weights(weights_0)
        model.reset_metrics()
        model.reset_states()
        opponent_history[:] = []
        self_history[:] = []
        n0[0] = next(nn)[1]
    nhist = len(opponent_history)
    if nhist > 1:
        if len(opponent_history) == 399:
            print(''.join(opponent_history))
            print(''.join(self_history))
        n = n0[0]
        x0 = rps_to_int(opponent_history[-min(nhist, n):])
        x1 = rps_to_int(self_history[-min(nhist, n):])
        y = [0.0, 0.0, 0.0]
        y[x0[-1]] = 1.0
        dataset = tf.data.Dataset.from_tensors(((np.array([x0[:-1]]), np.array([x1[:-1]])), np.array([y])))
        model.fit(dataset, epochs = 8, verbose = 0)
        guess = rps[(1 + np.argmax(model.predict((np.array([x0]), np.array([x1])), verbose = 0))) % 3]
    else:
        guess = np.random.choice(rps)
    self_history.append(guess)
    return guess
#play(player, quincy, 400)

In [ ]:
# This approach is hybrid of above 2 but doesn't work well enough
rps = ('R', 'P', 'S')
rps2int = {'R' : 0, 'P' : 1, 'S' : 2, '' : 3}
int2rps = {rps2int[c] : c for c in rps2int}
rps_to_int = lambda hist : [rps2int[c] for c in hist]

inp0 = tf.keras.Input(shape = (None,), batch_size = 1)
x0 = tf.keras.layers.Embedding(input_dim = 3, output_dim = 2, mask_zero = False)(inp0)
x0 = tf.keras.layers.LSTM(units = 1, stateful = False, recurrent_initializer = 'glorot_uniform')(x0)
inp1 = tf.keras.Input(shape = (None,), batch_size = 1)
x1 = tf.keras.layers.Embedding(input_dim = 3, output_dim = 2, mask_zero = False)(inp1)
x1 = tf.keras.layers.LSTM(units = 1, stateful = False, recurrent_initializer = 'glorot_uniform')(x1)
x2 = tf.keras.layers.concatenate([x0, x1])
x2 = tf.keras.layers.Dense(units = 3, kernel_initializer = 'glorot_uniform')(x2)
model = tf.keras.Model(inputs = (inp0, inp1), outputs = x2)
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),    # this loss function works because it's applied across the last dimension of the predictions
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.002))
weights_0 = model.get_weights()
nn = enumerate([6, 3, 3, 11])

def player(prev_play, opponent_history=[], self_history=[], n0 = [6]):
    opponent_history.append(prev_play)
    if opponent_history[-1] == '':
        tf.keras.backend.clear_session()
        model.set_weights(weights_0)
        model.reset_metrics()
        model.reset_states()
        opponent_history[:] = []
        self_history[:] = []
        n0[0] = next(nn)[1]
    nhist = len(opponent_history)
    if nhist > 1:
        if len(opponent_history) == 399:
            print(''.join(opponent_history))
            print(''.join(self_history))
        n = n0[0]
        x0s = [rps_to_int(opponent_history[i * n : (i+1) * n]) for i in range(len(opponent_history) // n)]
        x0s.append(rps_to_int(opponent_history[-n:]))
        x1s = [rps_to_int(self_history[i * n : (i+1) * n]) for i in range(len(self_history) // n)]
        x1s.append(rps_to_int(self_history[-n:]))
        ys = [[0.0, 0.0, 0.0] for _ in x0s]
        for i in range(len(x0s)):
            xx = x0s[i].pop()
            ys[i][xx] = 1.0
            dataset = tf.data.Dataset.from_tensors(((np.array([x0s[i]]), np.array([x1s[i]])), np.array([ys[i]])))
            model.fit(dataset, epochs = 1, verbose = 0)
        guess = rps[(1 + np.argmax(model.predict((np.array([x0s[-1][1:] + [xx]]), np.array([x1s[-1]])), verbose = 0))) % 3]
    else:
        guess = np.random.choice(rps)
    self_history.append(guess)
    return guess
#play(player, abbey, 400)

In [ ]:
model.summary()